In [6]:
#mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
!ls

gdrive	sample_data


In [0]:
# coding: utf-8
import tensorflow as tf

In [0]:
#data padding and batching

MAX_LEN = 50 #maximum words in one line
SOS_ID = 1 #ID of the start of line

#read language data from file
#convert to tensorflow datasets
def Makedataset(file_path):
  dataset = tf.data.TextLineDataset(file_path)
  #convert dataset to a single dimension vector 
  dataset = dataset.map(lambda string: tf.string_split([string]).values)
  #convert the data stream from string to integer
  dataset = dataset.map(lambda string: tf.string_to_number(string, tf.int32))
  #get the number of words in each line and add the number to the dataset
  dataset = dataset.map(lambda x: (x, tf.size(x)))
  return dataset


#add padding and batching to the source dataset and target dataset
def MakeSrcTrgDataset(src_path, trg_path, batch_size):
  #read the source data and target data to tf datasets
  src_data = Makedataset(src_path)
  trg_data = Makedataset(trg_path)
  #combine two dataset to one dataset
  # using zip() 
  # data[0][0] : source sentence
  # data[0][1] : source sentence length
  # data[1][0] : target sentence
  # data[1][1] : target sentence length
  dataset = tf.data.Dataset.zip((src_data, trg_data))
  
  #dataset filtering
  #delete empty sentence or end only with <eos>
  #as well as very long sentences (length > 50 words)
  #source: https://www.tensorflow.org/api_docs/python/tf/data/Dataset 
  def FilterLength(src_tuple, trg_tuple):
    ((src_input, src_len), (trg_input, trg_len)) = (src_tuple, trg_tuple)
    
    src_len_ok = tf.logical_and(
        tf.greater(src_len, 1), tf.less_equal(src_len, MAX_LEN))
    trg_len_ok = tf.logical_and(
        tf.greater(trg_len, 1), tf.less_equal(trg_len, MAX_LEN))
    return tf.logical_and(src_len_ok, trg_len_ok)
  dataset = dataset.filter(FilterLength) 
  
  #we need data format as input to the encoder as "<sos> a b c"
  #and uotput format as "a b c <eos>"
  #chang the dataset format as "a b c <eos>"
  def MakeTrgInput(src_tuple, trg_tuple):
    ((src_input, src_len), (trg_label, trg_len)) = (src_tuple, trg_tuple)
    trg_input = tf.concat([[SOS_ID], trg_label[:-1]], axis=0)
    return ((src_input, src_len), (trg_input, trg_label, trg_len))
    
  dataset = dataset.map(MakeTrgInput)
  
  #random shufful prevent overfitting
  dataset = dataset.shuffle(10000)
  
  #define padding shape and size
  padded_shape = (
    (tf.TensorShape([None]), #source sentence lenth unknown
    tf.TensorShape([])),     #length is integer
    (tf.TensorShape([None]), #target sentence (decoder input) length unknown
    tf.TensorShape([None]),  #target sentence (decoder output) length unknown
    tf.TensorShape([])))     #target sentence length is integer
  
  #use tensorflow padded_batch function to batching the data
  batched_dataset = dataset.padded_batch(batch_size, padded_shape)
  return batched_dataset

In [0]:
#test functions
src_data = Makedataset(train_data)
iterator = src_data.make_one_shot_iterator().get_next()

dataset = MakeSrcTrgDataset(train_data, target_data, 20)
iterator2 = src_data.make_one_shot_iterator().get_next()

with tf.Session() as session:
  print(session.run(iterator))  
  print(session.run(iterator2))  
  print(session.run(iterator2))  

  


NameError: ignored

In [0]:
#variables:

SRC_TRAIN_DATA = "/content/gdrive/My Drive/nlp_project/en-zh_data/train_en.txt" #english training data file
TRG_TRAIN_DATA = "/content/gdrive/My Drive/nlp_project/en-zh_data/train_zh.txt" #mandarine training data file

CHECKPOINT_PATH = "/content/gdrive/My Drive/nlp_project/en-zh_data/seq2seq_ckpt" #checkpoint path to save trained network

HIDDEN_SIZE = 1024              #LSTM hidden layer size
NUM_LAYERS = 2                  #number of LSTM RNN layer
SRC_VOCAB_SIZE = 10000          #english vocabulary size
TRG_VOCAB_SIZE = 5000           #mandarine vocabulary size
BATCH_SIZE = 100                #training batch size
NUM_EPOCH = 100                 #number of epoch
KEEP_PROB = 0.8                 #probability of dropout nodes
MAX_GRAD_NORM = 5               #control gradient limit
SHARE_EMB_AND_SOFTMAX = True    #share parameters between softmax layer and emmbeding layer

In [0]:

#define translation model
class NMTModel(object):
  #initialize variables needed in the model
  def __init__(self):
    #define encoder and decoder's LSTM structure
    self.enc_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) for _ in range(NUM_LAYERS)])
    self.dec_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) for _ in range(NUM_LAYERS)])
    #define word embedding for source and target language
    self.src_embedding = tf.get_variable("src_emb", [SRC_VOCAB_SIZE, HIDDEN_SIZE])
    self.trg_embedding = tf.get_variable("trg_emb", [TRG_VOCAB_SIZE, HIDDEN_SIZE])
    
    #define softmax layer variables
    if SHARE_EMB_AND_SOFTMAX:
      self.softmax_weight = tf.transpose(self.trg_embedding)
    else:
      self.softmax_bias = tf.get_variable("weight", [HIDDED_SIZE, TRG_VOCAB_SIZE])
    self.softmax_bias = tf.get_variable("softmax_bias", [TRG_VOCAB_SIZE])
    
    # defin src_input, src_size, trg_input, trg_label, trg_size
    # from MakeSrcTrgDataset 
  def forward(self, src_input, src_size, trg_input, trg_label, trg_size):
      batch_size = tf.shape(src_input)[0]

      #word embedding
      src_emb = tf.nn.embedding_lookup(self.src_embedding, src_input)
      trg_emb = tf.nn.embedding_lookup(self.trg_embedding, trg_input)

      #dropout。
      src_emb = tf.nn.dropout(src_emb, KEEP_PROB)
      trg_emb = tf.nn.dropout(trg_emb, KEEP_PROB)

      # construct encoder
      with tf.variable_scope("encoder"):
          enc_outputs, enc_state = tf.nn.dynamic_rnn(
              self.enc_cell, src_emb, src_size, dtype=tf.float32)


      # construct decoder
      with tf.variable_scope("decoder"):
          dec_outputs, _ = tf.nn.dynamic_rnn(
              self.dec_cell, trg_emb, trg_size, initial_state=enc_state)

      # use log perplexity as loss function
      output = tf.reshape(dec_outputs, [-1, HIDDEN_SIZE])
      logits = tf.matmul(output, self.softmax_weight) + self.softmax_bias
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
          labels=tf.reshape(trg_label, [-1]), logits=logits)

      # calculation of average cost
      label_weights = tf.sequence_mask(
          trg_size, maxlen=tf.shape(trg_label)[1], dtype=tf.float32)
      label_weights = tf.reshape(label_weights, [-1])
      cost = tf.reduce_sum(loss * label_weights)
      cost_per_token = cost / tf.reduce_sum(label_weights)

      # back propergation of variables
      trainable_variables = tf.trainable_variables()

      # defin optimization and training steps
      grads = tf.gradients(cost / tf.to_float(batch_size),
                           trainable_variables)
      grads, _ = tf.clip_by_global_norm(grads, MAX_GRAD_NORM)
      optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
      train_op = optimizer.apply_gradients(
          zip(grads, trainable_variables))
      return cost_per_token, train_op

In [9]:
# train model in epoch
def run_epoch(session, cost_op, train_op, saver, step):
    # single epoch。
    # repeats until train all the data from the dataset
    while True:
        try:
            # run train_op with loss
            cost, _ = session.run([cost_op, train_op])
            if step % 1 == 0:
                print("After %d steps, per token cost is %.3f" % (step, cost))
            # save to checkpoint in every 200 steps
            if step % 200 == 0:
                saver.save(session, CHECKPOINT_PATH, global_step=step)
            step += 1
        except tf.errors.OutOfRangeError:
            break
    return step
def main():
    #reset tensorflow graph
    tf.reset_default_graph() 
    # defin initialization variables
    initializer = tf.random_uniform_initializer(-0.05, 0.05)

    # defin training nmt model
    with tf.variable_scope("nmt_model", reuse=None, 
                           initializer=initializer):
        train_model = NMTModel()
  
    # defin input data
    data = MakeSrcTrgDataset(SRC_TRAIN_DATA, TRG_TRAIN_DATA, BATCH_SIZE)
    iterator = data.make_initializable_iterator()
    (src, src_size), (trg_input, trg_label, trg_size) = iterator.get_next()
 
    # Define a forward calculation graph. The input data is supplied to the forward function in tensor form.
    cost_op, train_op = train_model.forward(src, src_size, trg_input,
                                            trg_label, trg_size)
   
  # train model。
    my_ckpt= "/content/gdrive/My Drive/nlp_project/en-zh_data/seq2seq_ckpt-800.data-00000-of-00001" #english training data file
    saver = tf.train.Saver()
    step = 0
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(NUM_EPOCH):
            print("In iteration: %d" % (i + 1))
            sess.run(iterator.initializer)
            step = run_epoch(sess, cost_op, train_op, saver, step)
if __name__ == "__main__":
    main() 

NameError: ignored

In [0]:
import tensorflow as tf
import codecs
import sys

In [0]:
# checkpoint path 
CHECKPOINT_PATH = "/content/gdrive/My Drive/nlp_project/en-zh_data/attention_seq2seq_ckpt-61600" #checkpoint path to save trained network

# hyper parameters
HIDDEN_SIZE = 1024                        
DECODER_LAYERS  = 2                       
SRC_VOCAB_SIZE = 10000                   
TRG_VOCAB_SIZE = 5000                    
SHARE_EMB_AND_SOFTMAX = True          

# Vocabulary files
SRC_VOCAB = "/content/gdrive/My Drive/nlp_project/en-zh_data/train_en.vocab" 
TRG_VOCAB = "/content/gdrive/My Drive/nlp_project/en-zh_data/train_zh.vocab"

# id of <sos> and <eos>
SOS_ID = 1
EOS_ID = 2

In [0]:
#define translation model
class NMTModel(object):
    #initialize variables needed in the model
    def __init__(self):
      #define encoder and decoder's LSTM structure
        self.enc_cell_fw = tf.keras.layers.LSTMCell(HIDDEN_SIZE)
        self.enc_cell_bw = tf.keras.layers.LSTMCell(HIDDEN_SIZE)
        self.dec_cell = tf.nn.rnn_cell.MultiRNNCell(
          [tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
           for _ in range(DECODER_LAYERS)])

        # word embedding   
        self.src_embedding = tf.get_variable(
            "src_emb", [SRC_VOCAB_SIZE, HIDDEN_SIZE])
        self.trg_embedding = tf.get_variable(
            "trg_emb", [TRG_VOCAB_SIZE, HIDDEN_SIZE])

        # softmax layer
        if SHARE_EMB_AND_SOFTMAX:
            self.softmax_weight = tf.transpose(self.trg_embedding)
        else:
            self.softmax_weight = tf.get_variable(
               "weight", [HIDDEN_SIZE, TRG_VOCAB_SIZE])
        self.softmax_bias = tf.get_variable(
            "softmax_bias", [TRG_VOCAB_SIZE])

    def inference(self, src_input):
        # conver input to a batch size of 1
        src_size = tf.convert_to_tensor([len(src_input)], dtype=tf.int32)
        src_input = tf.convert_to_tensor([src_input], dtype=tf.int32)
        src_emb = tf.nn.embedding_lookup(self.src_embedding, src_input)

        with tf.variable_scope("encoder"):
            # initialize encoder
            enc_outputs, enc_state = tf.nn.bidirectional_dynamic_rnn(
                self.enc_cell_fw, self.enc_cell_bw, src_emb, src_size, 
                dtype=tf.float32)
            enc_outputs = tf.concat([enc_outputs[0], enc_outputs[1]], -1)    
        
        with tf.variable_scope("decoder"):
            # initializa decoder
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                HIDDEN_SIZE, enc_outputs,
                memory_sequence_length=src_size)

            attention_cell = tf.contrib.seq2seq.AttentionWrapper(
                self.dec_cell, attention_mechanism,
                attention_layer_size=HIDDEN_SIZE)
   
        # defin maximum decode step to prevent looping
        MAX_DEC_LEN=100

        with tf.variable_scope("decoder/rnn/attention_wrapper"):
            # tensor arra to store sentence
            init_array = tf.TensorArray(dtype=tf.int32, size=0,
                dynamic_size=True, clear_after_read=False)
            # decoder input start with <sos>
            init_array = init_array.write(0, SOS_ID)
            init_loop_var = (
                attention_cell.zero_state(batch_size=1, dtype=tf.float32),
                init_array, 0)

            # use tf while loop untill shows <eos>
            def continue_loop_condition(state, trg_ids, step):
                return tf.reduce_all(tf.logical_and(
                    tf.not_equal(trg_ids.read(step), EOS_ID),
                    tf.less(step, MAX_DEC_LEN-1)))

            def loop_body(state, trg_ids, step):
                # read last step word embedding vector
                trg_input = [trg_ids.read(step)]
                trg_emb = tf.nn.embedding_lookup(self.trg_embedding,
                                                 trg_input)
                # call attention_cell to calculate a step backwords
                dec_outputs, next_state = attention_cell.call(
                    state=state, inputs=trg_emb)
                # get the output
                output = tf.reshape(dec_outputs, [-1, HIDDEN_SIZE])
                logits = (tf.matmul(output, self.softmax_weight)
                          + self.softmax_bias)
                next_id = tf.argmax(logits, axis=1, output_type=tf.int32)
                # conver source id to target language id
                trg_ids = trg_ids.write(step+1, next_id[0])
                return next_state, trg_ids, step+1

            # run tf.while_loop
            state, trg_ids, step = tf.while_loop(
                continue_loop_condition, loop_body, init_loop_var)
            return trg_ids.stack()

In [36]:
# clear curren graph
tf.reset_default_graph() 
def main():
    # defin NMT model
    with tf.variable_scope("nmt_model", reuse=None):
        model = NMTModel()

    # Testing sentence
    #test_en_text = "Today I 'm going to talk about a project in language translation . <eos>"
    #test_en_text = "The weather is good .  <eos>"
    #test_en_text = "write report takes a lot of time .  <eos>"
    #test_en_text = "The burger week is end . <eos>"
    
    
    
    
    
    
    test_en_text = "I can finish this report with some help<eos>"
    # convert ID to words
    with codecs.open(SRC_VOCAB, "r", "utf-8") as f_vocab:
        src_vocab = [w.strip() for w in f_vocab.readlines()]
        src_id_dict = dict((src_vocab[x], x) for x in range(len(src_vocab)))
    test_en_ids = [(src_id_dict[token] if token in src_id_dict else src_id_dict['<unk>'])
                   for token in test_en_text.split()]
    #print(test_en_ids)

    # construct decoder
    output_op = model.inference(test_en_ids)
    sess = tf.Session()
    saver = tf.train.Saver()
    saver.restore(sess, CHECKPOINT_PATH)

    # read translate result
    output_ids = sess.run(output_op)
    
  
    
    # convert target ID to target sentence
    with codecs.open(TRG_VOCAB, "r", "utf-8") as f_vocab:
        trg_vocab = [w.strip() for w in f_vocab.readlines()]
    output_text = ''.join([trg_vocab[x] for x in output_ids])
    
    # print result
    print('Source language:',test_en_text)
    print('Source number sequence:',test_en_ids)
    print('Target number sequence:',output_ids)
    print('Target language:',output_text.encode('utf8').decode(sys.stdout.encoding))
    sess.close()

if __name__ == "__main__":
    main()

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/nlp_project/en-zh_data/attention_seq2seq_ckpt-61600
Source language: I can finish this report with some help<eos>
Source number sequence: [11, 29, 1858, 19, 1563, 28, 96, 0]
Target number sequence: [  1   5  29  20 249  73  10  12 305 715   4 205 835  14   9  25  13 129
   5   3 585 274   6 585 274  13  25  16 585  89   6 585  89 395 205  73
  14   6   2]
Target language: <sos>我可以完成这个讲述，结束了一些有关我的报告。报告有些人报道。报道总结成了。<eos>
